In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from lightgbm import LGBMClassifier

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import SMOTE, ADASYN

from xgboost import XGBClassifier

In [2]:
path = 'C:\\Users\\swwho\\Desktop\\ai_competition\\AI-Competition/data/phone_cancle_prediction/'

In [3]:
train = pd.read_csv(path+'train.csv')

In [4]:
train.describe()

,가입일,음성사서함이용,주간통화시간,주간통화횟수,주간통화요금,저녁통화시간,저녁통화횟수,저녁통화요금,밤통화시간,밤통화횟수,밤통화요금,상담전화건수,전화해지여부
count,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000,30200.000000
mean,159.732649,12.037815,250.563457,159.175728,41.091447,263.247215,158.474536,25.394260,263.438285,157.047417,12.074648,2.288079,0.109868
std,123.752781,24.925334,84.403350,101.573057,23.179017,74.064922,102.200293,8.146126,73.778416,101.721601,9.414261,2.050746,0.312730
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,0.000000,20.800000,20.000000,1.040000,0.000000,0.000000
25%,63.000000,0.000000,222.800000,88.000000,24.400000,223.700000,87.000000,22.220000,223.700000,87.000000,4.700000,2.000000,0.000000
50%,118.000000,1.000000,234.000000,117.500000,30.800000,243.000000,112.000000,23.320000,242.800000,108.000000,9.080000,2.000000,0.000000
75%,228.000000,22.000000,289.600000,223.000000,45.900000,290.300000,223.000000,28.820000,290.500000,222.000000,20.290000,2.000000,0.000000
max,2212.000000,1112.000000,481.900000,489.000000,118.600000,481.600000,489.000000,49.980000,481.800000,490.000000,44.980000,11.000000,1.000000


In [5]:
train = pd.read_csv(path+'train.csv')
train['총통화시간'] = train['주간통화시간'] + train['저녁통화시간'] + train['밤통화시간']
train['총통화요금'] = train['주간통화요금'] + train['저녁통화요금'] + train['밤통화요금']
train['총통화횟수'] = train['주간통화횟수'] + train['저녁통화횟수'] + train['밤통화횟수']
train['분당요금'] = train['총통화요금'] / train['총통화시간']
train = train.loc[(train['밤통화시간'] <= 405)]
train = train.reset_index(drop=True)

In [6]:
def standard_scale(df, sc=StandardScaler(), test=False):
    scaler = sc
    cols = df.columns
    if not test:
        df = scaler.fit_transform(df)
        df = pd.DataFrame(df, columns=cols)
        return scaler, df
    else:
        df = scaler.transform(df)
        df = pd.DataFrame(df, columns=cols)
        return df

In [7]:
x = train.drop(['ID','전화해지여부'], axis=1)
y = train['전화해지여부']

scaler, x = standard_scale(x, StandardScaler())

train_x, val_x, train_y, val_y = train_test_split(x,y, test_size=0.05, stratify=y, random_state=123)

In [8]:
def f1(y_true, y_pred):
    y_pred = np.where(y_pred >= 0.5, 1, 0)
    score= f1_score(y_true, y_pred, average='macro')
    return 'score', score, False

In [9]:
lgb_model = LGBMClassifier(learning_rate=0.058, n_jobs=-1, n_estimators=900, random_state=123, class_weight='balanced')
lgb_model.fit(train_x, train_y, eval_metric=f1)

LGBMClassifier(class_weight='balanced', learning_rate=0.058, n_estimators=900,
               random_state=123)

In [10]:
xg_model = XGBClassifier(random_state=123, learning_rate=0.55, eval_metric=f1, n_estimators=1000)
xg_model.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=<function f1 at 0x0000022143248EE0>, gamma=0,
              gpu_id=-1, grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.55, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              predictor='auto', random_state=123, reg_alpha=0, reg_lambda=1, ...)

In [11]:
clf_preds = clf.predict(val_x)
f1_score(val_y, clf_preds, average='macro')

NameError: name 'clf' is not defined

## SUBMIT

In [ ]:
test = pd.read_csv(path+'test.csv')
test = test.drop(['ID'], axis=1)
test['총통화시간'] = test['주간통화시간'] + test['저녁통화시간'] + test['밤통화시간']
test['총통화요금'] = test['주간통화요금'] + test['저녁통화요금'] + test['밤통화요금']
test['총통화횟수'] = test['주간통화횟수'] + test['저녁통화횟수'] + test['밤통화횟수']
test['분당요금'] =  test['총통화요금'] /  test['총통화시간']
test = standard_scale(test, scaler, True)
test_preds = lgb_model.predict(test)

In [ ]:
submit = pd.read_csv(path+'sample_submission.csv')
submit['전화해지여부'] = test_preds
submit

In [ ]:
submit.to_csv('sumbit.csv', index=False)